In [ ]:
import io
import json
import logging
import os
import re
import unicodedata

import pandas as pd
from minio import Minio
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, lit

In [ ]:
minio_connection = ""  

In [ ]:
try:
    minio_conn = json.loads(minio_connection)
except json.JSONDecodeError:
    with open('../variables/minio_connection.json', "r") as minio_connection_file:
        minio_conn = json.loads(minio_connection_file.read())

In [ ]:
class LazySparkSession:
    packages = [
        "io.delta:delta-spark_2.13:4.0.0",
        "org.apache.hadoop:hadoop-aws:3.4.0",
        "com.amazonaws:aws-java-sdk-bundle:1.12.787",
    ]

    def __init__(self, access_key, secret_key, endpoint):
        self._access_key = access_key
        self._secret_key = secret_key
        self._endpoint = endpoint
        

    def start(
        self,
        app_name: str = "Airflow Spark Delta Minio App",
        executor_memory: str = "1g",
        driver_memory: str = "1g",
        driver_maxresultsize: str = "1g",
        master_url: str = "local[*]",
    ):

        builder = (
            SparkSession
            .Builder()
            .appName("XLSX to Delta Bronze")
            .config("spark.hadoop.fs.s3a.access.key", self._access_key)
            .config("spark.hadoop.fs.s3a.secret.key", self._secret_key)
            .config("spark.hadoop.fs.s3a.endpoint", self._endpoint)
            .config("spark.hadoop.delta.enableFastS3AListFrom", "true")
            #
            .config("spark.executor.memory", executor_memory)
            .config("spark.driver.memory", driver_memory)
            .config("spark.driver.maxResultSize", driver_maxresultsize)
            #
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            #
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
            #
            .config("spark.jars.packages", ",".join(self.packages))
            .config("spark.ui.port", "0")
            .master(master_url)
            
        )

        return builder.getOrCreate()

In [ ]:
spark = LazySparkSession(
    access_key=minio_conn.get("access_key"), 
    secret_key=minio_conn.get("key"), 
    endpoint=minio_conn.get("endpoint")
).start()

In [ ]:
endpoint_raw = minio_conn["endpoint"]
endpoint_sem_http = endpoint_raw.replace("http://", "").replace("https://", "")

s3_client = Minio(
    endpoint=endpoint_sem_http,
    access_key=minio_conn['access_key'],
    secret_key=minio_conn['key'],
    secure=endpoint_raw.startswith("https")
)

In [ ]:
# Configuração básica de logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

In [ ]:
# Variáveis do projeto
bucket_landing = "landing"
bucket_bronze = "bronze"
prefix_landing = "anp/serie_levantamento_precos/"

In [ ]:
# Limpar e padronizar nome das colunas
def limpar_nome_colunas_spark(spark_df):
    """
    Limpa e padroniza as colunas do Dataframe.
    Remove acentos.
    Substitui caracteres não alfanumericos por underscore (_).
    Remove múltiplos underscores.
    """

    novas_colunas=[]
    for coluna in spark_df.columns:
        coluna_sem_acento = unicodedata.normalize("NFKD", coluna).encode("ASCII", "ignore").decode("ASCII")
        coluna_limpa = re.sub(r'[^a-zA-Z0-9_-]', '-', coluna_sem_acento)
        coluna_lower = coluna_limpa.lower()
        coluna_final = re.sub(r'_+', '_', coluna_lower).strip('_')
        novas_colunas.append(coluna_final)

    # Renomeia todas as colunas de uma vez
    return spark_df.toDF(*novas_colunas)

In [ ]:
# Listar arquivos .xlsx
def listar_arquivos_xlsx(bucket: str, prefix: str) -> list:
    try:
        arquivos = s3_client.list_objects(bucket, prefix=prefix, recursive=True)
        lista = [obj.object_name for obj in arquivos if obj.object_name.endswith(".xlsx")]
        logging.info(f"{len(lista)} arquivos .xlsx encontrados em s3://{bucket}/{prefix}")
        return lista
    except Exception as e:
        logging.error(f"Erro ao listar arquivos: {e}")
        return []

In [ ]:
# Leitura específica para este formato de arquivo
def ler_arquivos_anp(bucket: str, caminho_arquivo: str) -> pd.DataFrame:
    try:
        logging.info(f"Lendo arquivo com layout específico: {caminho_arquivo}")
        resposta = s3_client.get_object(bucket, caminho_arquivo)

        # Parâmetros descobertos na inspeção
        df = pd.read_excel(
            io.BytesIO(resposta.read()),
            engine="openpyxl",
            header=11,              # Cabeçalho indice 11
            skipfooter=12           # Conteúdo começa indice 12
        )
        
        # Remove colunas que foram criadas sem nome
        df.dropna(axis=1, how='all', inplace=True)

        return df
    except Exception as e:
        logging.error(f"Erro ao ler arquivo {caminho_arquivo}: {e}")
        raise

In [ ]:
# Salvar como Delta
def salvar_como_delta(spark_df, nome_arquivo: str):
    nome_limpo = os.path.splitext(os.path.basename(nome_arquivo))[0]
    nome_limpo_sanitizado = re.sub(r'[^a-zA-Z0-9_-]', '-', nome_limpo)

    caminho_bronze = f"s3a://{bucket_bronze}/anp/precos-combustiveis/{nome_limpo_sanitizado}"

    df_final = (
        spark_df.withColumn("_carga_data", current_timestamp())
                .withColumn("_arquivo_origem", lit(nome_arquivo))
    )

    df_final.write.format("delta").mode("overwrite").save(caminho_bronze)
    logging.info(f"Arquivo salvo com sucesso: {caminho_bronze}")

In [ ]:
# --- Execução principal
arquivos = listar_arquivos_xlsx(bucket_landing, prefix_landing)

for arquivo in arquivos:
    try:
        df_pandas = ler_arquivos_anp(bucket_landing, arquivo)
        df_spark = spark.createDataFrame(df_pandas)
        df_spark_limpo = limpar_nome_colunas_spark(df_spark)
        salvar_como_delta(df_spark_limpo, os.path.basename(arquivo))
    except Exception as e:
        logging.error(f"Erro ao processar {arquivo}: {e}")